In [14]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import train_test_split 
from datetime import datetime, timedelta
import xgboost as xgb 
import tensorflow as tf 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Embedding
from sklearn.model_selection import train_test_split

### Simplified CSM and FCFS model

In [4]:
class IFRS17Model:
    def __init__(self, premium, claims, expenses, risk_free_rate, illiquidity_premium, own_credit_risk, cost_of_capital_rate, coverage_period_years):
        self.premium = premium
        self.claims = claims
        self.expenses = expenses
        self.discount_rate = self.estimate_discount_rate(risk_free_rate, illiquidity_premium, own_credit_risk)
        self.cost_of_capital_rate = cost_of_capital_rate
        self.coverage_period_years = coverage_period_years

    def estimate_discount_rate(self, risk_free_rate, illiquidity_premium, own_credit_risk):

        return risk_free_rate + illiquidity_premium + own_credit_risk

    def project_cashflows(self):

        projected_premium = [self.premium] * self.coverage_period_years
        projected_claims = [self.claims] * self.coverage_period_years
        projected_expenses = [self.expenses] * self.coverage_period_years

        return {
        'premium': projected_premium,
        'claims': projected_claims,
        'expenses': projected_expenses
         }

    def discount_cashflows(self, cashflows):

        discounted_cashflows = [
        cashflow / ((1 + self.discount_rate) ** period) 
        for period, cashflow in enumerate(cashflows)
        ]
        return discounted_cashflows

    def calculate_risk_adjustment(self, cashflows):

        capital_required = np.std(cashflows) # Assume required capital is related to standard deviation
        risk_adjustment = sum([
        (capital_required * self.cost_of_capital_rate) / ((1 + self.discount_rate) ** period)
        for period in range(1, self.coverage_period_years + 1)
        ])
        return risk_adjustment

    def calculate_fulfilment_cashflows(self, projected_cashflows):

        discounted_premium = self.discount_cashflows(projected_cashflows['premium'])
        discounted_claims = self.discount_cashflows(projected_cashflows['claims'])
        discounted_expenses = self.discount_cashflows(projected_cashflows['expenses'])

        risk_adjustment = self.calculate_risk_adjustment(discounted_claims)

        fulfilment_cashflows = {
        'discounted_premium': discounted_premium,
        'discounted_claims': discounted_claims,
        'discounted_expenses': discounted_expenses,
        'risk_adjustment': risk_adjustment,
        }

        return fulfilment_cashflows

    def calculate_contractual_service_margin(self, fulfilment_cashflows):

        total_discounted_premium = np.sum(fulfilment_cashflows['discounted_premium'])
        total_discounted_claims = np.sum(fulfilment_cashflows['discounted_claims'])
        total_discounted_expenses = np.sum(fulfilment_cashflows['discounted_expenses'])
        risk_adjustment = fulfilment_cashflows['risk_adjustment']

        csm = total_discounted_premium - total_discounted_claims - total_discounted_expenses - risk_adjustment
        return csm

    def estimate(self):

        projected_cashflows = self.project_cashflows()
        fulfilment_cashflows = self.calculate_fulfilment_cashflows(projected_cashflows)
        csm = self.calculate_contractual_service_margin(fulfilment_cashflows)

        return {
        'fulfilment_cashflows': fulfilment_cashflows,
        'contractual_service_margin': csm
        }


ifrs17_model = IFRS17Model(
premium=1000,
claims=700,
expenses=100,
risk_free_rate=0.02, # 2% risk-free rate
illiquidity_premium=0.01, # 1% illiquidity premium
own_credit_risk=0.005, # 0.5% own credit risk
cost_of_capital_rate=0.06, # 6% cost of capital rate
coverage_period_years=10
)

results = ifrs17_model.estimate()
print("Fulfilment Cashflows:", results['fulfilment_cashflows'])
print("Contractual Service Margin (CSM):", results['contractual_service_margin'])

Fulfilment Cashflows: {'discounted_premium': [1000.0, 966.1835748792271, 933.5107003664031, 901.9427056680224, 871.4422276985724, 841.9731668585242, 813.500644307753, 785.9909606838193, 759.4115562162506, 733.730972189614], 'discounted_claims': [700.0, 676.328502415459, 653.4574902564822, 631.3598939676157, 610.0095593890006, 589.381216800967, 569.450451015427, 550.1936724786735, 531.5880893513754, 513.6116805327298], 'discounted_expenses': [100.0, 96.61835748792271, 93.35107003664031, 90.19427056680223, 87.14422276985724, 84.19731668585241, 81.35006443077529, 78.59909606838193, 75.94115562162506, 73.3730972189614], 'risk_adjustment': 29.67782257199374}
Contractual Service Margin (CSM): 1691.8594792016422


### Complex risk adjustment and discount rate calculation incorporated

In [10]:
np.random.seed(42)
historical_data = pd.DataFrame({
    'time': np.arange(1, 21),
    'interest_rate': np.random.uniform(0.01, 0.05, 20),
    'risk_adjustment': np.random.uniform(0.01, 0.02, 20)
})

start_date = datetime(2024, 1, 1) 
dates = [start_date + timedelta(days=30*i) for i in range(10)] 
premiums = [1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450] 
claims = [700, 710, 720, 730, 740, 750, 760, 770, 780, 790] 
expenses = [100, 105, 110, 115, 120, 125, 130, 135, 140, 145]

cashflows_data = pd.DataFrame({
    'date': dates,
    'premium': premiums,
    'claim': claims,
    'expense': expenses
})

def prepare_lstm_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 3
interest_rate_data = historical_data['interest_rate'].values
X, y = prepare_lstm_data(interest_rate_data, n_steps)

X = X.reshape((X.shape[0], X.shape[1], 1))

lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X, y, epochs=200, verbose=0)

def predict_discount_rate(model, data, n_steps):
    input_seq = np.array(data[-n_steps:])
    input_seq = input_seq.reshape((1, n_steps, 1))
    predicted_rate = model.predict(input_seq, verbose=0)
    return predicted_rate[0][0]

predicted_discount_rate = predict_discount_rate(lstm_model, interest_rate_data, n_steps)
print(f"Predicted Discount Rate: {predicted_discount_rate}")

X = historical_data[['time']] 
y = historical_data['risk_adjustment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42) 
gb_model.fit(X_train, y_train)
predicted_risk_adjustment = gb_model.predict([[historical_data['time'].values[-1] + 1]])[0]

print(f"Predicted Risk Adjustment Factor: {predicted_risk_adjustment}")


class IFRS17Model:
    def __init__(self, cashflows_data, discount_rate, risk_adjustment):
        self.cashflows_data = cashflows_data
        self.discount_rate = discount_rate
        self.risk_adjustment = risk_adjustment

    def project_cashflows(self):
        """
        Projects cashflows over the coverage period.

        Returns:
        dict: Projected premium, claims, and expenses cashflows.
        """
        return {
            'premium': self.cashflows_data['premium'].tolist(),
            'claims': self.cashflows_data['claim'].tolist(),
            'expenses': self.cashflows_data['expense'].tolist(),
            'dates': self.cashflows_data['date'].tolist()
        }

    def discount_cashflows(self, cashflows, dates):
        discounted_cashflows = []
        start_date = dates[0]
        for cashflow, date in zip(cashflows, dates):
            t = (date - start_date).days / 365.25 
            discounted_cashflow = cashflow / ((1 + self.discount_rate) ** t)
            discounted_cashflows.append(discounted_cashflow)
        return discounted_cashflows

    def calculate_fulfilment_cashflows(self, projected_cashflows):
        discounted_premium = self.discount_cashflows(projected_cashflows['premium'], projected_cashflows['dates'])
        discounted_claims = self.discount_cashflows(projected_cashflows['claims'], projected_cashflows['dates'])
        discounted_expenses = self.discount_cashflows(projected_cashflows['expenses'], projected_cashflows['dates'])

        fulfilment_cashflows = {
            'discounted_premium': discounted_premium,
            'discounted_claims': discounted_claims,
            'discounted_expenses': discounted_expenses,
            'risk_adjustment': self.risk_adjustment,
        }

        return fulfilment_cashflows

    def calculate_contractual_service_margin(self, fulfilment_cashflows):
        total_discounted_premium = np.sum(fulfilment_cashflows['discounted_premium'])
        total_discounted_claims = np.sum(fulfilment_cashflows['discounted_claims'])
        total_discounted_expenses = np.sum(fulfilment_cashflows['discounted_expenses'])
        risk_adjustment = fulfilment_cashflows['risk_adjustment']

        csm = total_discounted_premium - total_discounted_claims - total_discounted_expenses - risk_adjustment
        return csm

    def estimate(self):
        projected_cashflows = self.project_cashflows()
        fulfilment_cashflows = self.calculate_fulfilment_cashflows(projected_cashflows)
        csm = self.calculate_contractual_service_margin(fulfilment_cashflows)

        return {
            'fulfilment_cashflows': fulfilment_cashflows,
            'contractual_service_margin': csm
        }
 

ifrs17_model = IFRS17Model(
    cashflows_data=cashflows_data,
    discount_rate=predicted_discount_rate,
    risk_adjustment=predicted_risk_adjustment
)

results = ifrs17_model.estimate()
print("Fulfilment Cashflows:", results['fulfilment_cashflows']) 
print("Contractual Service Margin (CSM):", results['contractual_service_margin'])


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Predicted Discount Rate: 0.026608966290950775
Predicted Risk Adjustment Factor: 0.014447915026203073
Fulfilment Cashflows: {'discounted_premium': [1000.0, 1047.737622719998, 1095.2648876818575, 1142.58247975649, 1189.6910818400845, 1236.5913748594369, 1283.284037777268, 1329.7697475975244, 1376.0491793706685, 1422.123006198953], 'discounted_claims': [700.0, 708.4702020297128, 716.9006537553977, 725.291487149772, 733.6428338013854, 741.9548249156621, 750.2275913159413, 758.4612634445139, 766.6559713636582, 774.8118447566709], 'discounted_expenses': [100.0, 104.77376227199979, 109.52648876818574, 114.25824797564901, 118.96910818400845, 123.65913748594369, 128.3284037777268, 132.97697475975244, 137.60491793706683, 142.2123006198953], 'risk_adjustment': 0.014447915026203073}
Contractual Service Margin (CSM): 3534.352955574312


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


### Complex risk adjustment and discount rate calculation incorporated, with more realistic ML techniques

In [18]:
np.random.seed(42)
historical_data = pd.DataFrame({
    'time': np.arange(1, 21),
    'interest_rate': np.random.uniform(0.01, 0.05, 20),
    'risk_adjustment': np.random.uniform(0.01, 0.02, 20)
})
 
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=30*i) for i in range(10)] 
premiums = [1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450] 
claims = [700, 710, 720, 730, 740, 750, 760, 770, 780, 790] 
expenses = [100, 105, 110, 115, 120, 125, 130, 135, 140, 145]

cashflows_data = pd.DataFrame({
    'date': dates,
    'premium': premiums,
    'claim': claims,
    'expense': expenses
})


def prepare_transformer_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 3
interest_rate_data = historical_data['interest_rate'].values
X, y = prepare_transformer_data(interest_rate_data, n_steps)

class TransformerTimeSeries:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = self.build_model()

    def build_model(self):
        inputs = Input(shape=self.input_shape)
        x = Embedding(input_dim=50, output_dim=64)(inputs)
        x = LayerNormalization()(x)
        x = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
        x = Dropout(0.1)(x)
        x = Dense(1)(x)
        model = Model(inputs=inputs, outputs=x)
        model.compile(optimizer='adam', loss='mse')
        return model

    def fit(self, X, y, epochs=200):
        self.model.fit(X, y, epochs=epochs, verbose=0)

    def predict(self, X):
        return self.model.predict(X, verbose=0)

X = X.reshape((X.shape[0], X.shape[1], 1))
transformer_model = TransformerTimeSeries(input_shape=(n_steps, 1))
transformer_model.fit(X, y, epochs=200)

def predict_discount_rate(model, data, n_steps):
    input_seq = np.array(data[-n_steps:])
    input_seq = input_seq.reshape((1, n_steps, 1))
    predicted_rate = model.predict(input_seq)
    return predicted_rate[0][0]

predicted_discount_rate = predict_discount_rate(transformer_model, interest_rate_data, n_steps)
print(f"Predicted Discount Rate: {predicted_discount_rate}")

 
X = historical_data[['time']] 
y = historical_data['risk_adjustment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42) 
xgb_model.fit(X_train, y_train)
predicted_risk_adjustment = xgb_model.predict([[historical_data['time'].values[-1] + 1]])[0] 
print(f"Predicted Risk Adjustment Factor: {predicted_risk_adjustment}")


class IFRS17Model:
    def __init__(self, cashflows_data, discount_rate, risk_adjustment):
        self.cashflows_data = cashflows_data
        self.discount_rate = discount_rate
        self.risk_adjustment = risk_adjustment

    def project_cashflows(self):
        return {
            'premium': self.cashflows_data['premium'].tolist(),
            'claims': self.cashflows_data['claim'].tolist(),
            'expenses': self.cashflows_data['expense'].tolist(),
            'dates': self.cashflows_data['date'].tolist()
        }

    def discount_cashflows(self, cashflows, dates):
        discounted_cashflows = []
        start_date = dates[0]
        for cashflow, date in zip(cashflows, dates):
            t = (date - start_date).days / 365.25  
            discounted_cashflow = cashflow / ((1 + self.discount_rate) ** t)
            discounted_cashflows.append(discounted_cashflow)
        return discounted_cashflows

    def calculate_fulfilment_cashflows(self, projected_cashflows):
        discounted_premium = self.discount_cashflows(projected_cashflows['premium'], projected_cashflows['dates'])
        discounted_claims = self.discount_cashflows(projected_cashflows['claims'], projected_cashflows['dates'])
        discounted_expenses = self.discount_cashflows(projected_cashflows['expenses'], projected_cashflows['dates'])

        fulfilment_cashflows = {
            'discounted_premium': discounted_premium,
            'discounted_claims': discounted_claims,
            'discounted_expenses': discounted_expenses,
            'risk_adjustment': self.risk_adjustment,
        }

        return fulfilment_cashflows

    def calculate_contractual_service_margin(self, fulfilment_cashflows):
        total_discounted_premium = np.sum(fulfilment_cashflows['discounted_premium'])
        total_discounted_claims = np.sum(fulfilment_cashflows['discounted_claims'])
        total_discounted_expenses = np.sum(fulfilment_cashflows['discounted_expenses'])
        risk_adjustment = fulfilment_cashflows['risk_adjustment']

        csm = total_discounted_premium - total_discounted_claims - total_discounted_expenses - risk_adjustment
        return csm

    def estimate(self):
        projected_cashflows = self.project_cashflows()
        fulfilment_cashflows = self.calculate_fulfilment_cashflows(projected_cashflows)
        csm = self.calculate_contractual_service_margin(fulfilment_cashflows)

        return {
            'fulfilment_cashflows': fulfilment_cashflows,
            'contractual_service_margin': csm
        }

ifrs17_model = IFRS17Model(
    cashflows_data=cashflows_data,
    discount_rate=predicted_discount_rate,
    risk_adjustment=predicted_risk_adjustment
)

results = ifrs17_model.estimate()
print("Fulfilment Cashflows:", results['fulfilment_cashflows']) 
print("Contractual Service Margin (CSM):", results['contractual_service_margin'])



Predicted Discount Rate: [[0.02613571]]
Predicted Risk Adjustment Factor: 0.015065902844071388
Fulfilment Cashflows: {'discounted_premium': [array([[1000.]], dtype=float32), array([[1047.7773]], dtype=float32), array([[1095.3479]], dtype=float32), array([[1142.7123]], dtype=float32), array([[1189.8713]], dtype=float32), array([[1236.8256]], dtype=float32), array([[1283.5758]], dtype=float32), array([[1330.1224]], dtype=float32), array([[1376.4662]], dtype=float32), array([[1422.6078]], dtype=float32)], 'discounted_claims': [array([[700.]], dtype=float32), array([[708.4971]], dtype=float32), array([[716.95496]], dtype=float32), array([[725.3739]], dtype=float32), array([[733.754]], dtype=float32), array([[742.09534]], dtype=float32), array([[750.39813]], dtype=float32), array([[758.6624]], dtype=float32), array([[766.8883]], dtype=float32), array([[775.076]], dtype=float32)], 'discounted_expenses': [array([[100.]], dtype=float32), array([[104.77773]], dtype=float32), array([[109.53478]]